In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

pd.set_option('display.max_columns', 500)

In [3]:
susp_op = pd.read_csv("../data/susp_ops.csv")
off_op = pd.read_csv("../data/off_ops.csv")
off_mem = pd.read_csv("../data/off_members.csv")
susp_mem = pd.read_csv("../data/susp_members.csv")

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4,9,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2,3,8,9,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2,3,5,11,17,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [82]:
off_op.P_OPERATIONDATETIME = pd.to_datetime(off_op.P_OPERATIONDATETIME)

In [15]:
susp_op["label"] = susp_op["P_SENDTOKFMBOOL"].isin([1,5,7,8])

In [83]:
merged = off_op[to_take_off_op].merge(off_mem[to_take_off_mem], left_on="ID", right_on="P_OPERATIONID")

In [134]:
agg_cols = ["P_CURRENCYCODE", "P_EKNPCODE", "P_DOCCATEGORY", "P_COUNTRYCODE", "P_CLIENT_TYPE", 
            "P_CLIENTKIND", "P_BANK", "P_BANKCOUNTRYCODE"]

agg_funcs = [np.size, np.sum, np.mean]

def aggregate_for_period(group, dt, days):
    min_dt = dt - pd.DateOffset(days=days)
    group = group[(group.P_OPERATIONDATETIME >= min_dt) & (group.P_OPERATIONDATETIME <= dt)]
    
    in_group = group[group.P_CLIENTROLE == 2]
    out_group = group[group.P_CLIENTROLE == 1]
    
    name_format = "{days}days_{col}_{value}_{func}_{dir}"
    
    features = {}
    
    for dir, dir_group in [("in", in_group), ("out", out_group)]:
        for col in agg_cols:
            for func in agg_funcs:
                pivotted = dir_group[["P_BASEAMOUNT", col]].pivot_table(columns=col, values=["P_BASEAMOUNT"], aggfunc=func)

                for idx, value in enumerate(pivotted.columns):
                    feature_value = pivotted.iloc[0, idx]
                    name = name_format.format(days=days, col=col, value=value, func=func.__name__, dir=dir)
                    features[name] = feature_value
    return features

def get_features(group, dt):
    pass

aggregate_for_period(t, pd.to_datetime("2017-01-02"), 7)

{'7days_P_BANKCOUNTRYCODE_KZ_mean_in': 135597.4075,
 '7days_P_BANKCOUNTRYCODE_KZ_size_in': 8.0,
 '7days_P_BANKCOUNTRYCODE_KZ_sum_in': 1084779.26,
 '7days_P_BANK_EURIKZKA_mean_in': 135597.4075,
 '7days_P_BANK_EURIKZKA_size_in': 8.0,
 '7days_P_BANK_EURIKZKA_sum_in': 1084779.26,
 '7days_P_CLIENTKIND_55_mean_in': 135597.4075,
 '7days_P_CLIENTKIND_55_size_in': 8.0,
 '7days_P_CLIENTKIND_55_sum_in': 1084779.26,
 '7days_P_CLIENT_TYPE_1_mean_in': 135597.4075,
 '7days_P_CLIENT_TYPE_1_size_in': 8.0,
 '7days_P_CLIENT_TYPE_1_sum_in': 1084779.26,
 '7days_P_COUNTRYCODE_KZ_mean_in': 135597.4075,
 '7days_P_COUNTRYCODE_KZ_size_in': 8.0,
 '7days_P_COUNTRYCODE_KZ_sum_in': 1084779.26,
 '7days_P_CURRENCYCODE_398_mean_in': 135597.4075,
 '7days_P_CURRENCYCODE_398_size_in': 8.0,
 '7days_P_CURRENCYCODE_398_sum_in': 1084779.26,
 '7days_P_DOCCATEGORY_25_mean_in': 135597.4075,
 '7days_P_DOCCATEGORY_25_size_in': 8.0,
 '7days_P_DOCCATEGORY_25_sum_in': 1084779.26,
 '7days_P_EKNPCODE_421.0_mean_in': 43423.47,
 '7days_

Timestamp('2017-01-02 00:00:00')

In [105]:
"{days}days_{col}_{value}_{func}_{direction}".format(days=7, col="ID", value=421, func=np.mean.__name__, direction="in")

'7days_ID_421_mean_in'

In [129]:
t = merged.head(100000).groupby("P_ACCOUNT").get_group("KZ0294800KZT28600138")
t

,ID,P_CURRENCYCODE,P_OPERATIONDATETIME,P_BASEAMOUNT,P_EKNPCODE,P_DOCCATEGORY,P_COUNTRYCODE,P_CLIENT_TYPE,P_CLIENTROLE,P_CLIENTKIND,P_ACCOUNT,P_CLIENTID,P_BANK,P_BANKCOUNTRYCODE,P_OPERATIONID
43,528657550,398,2017-01-01,151075.01,423,25,KZ,1,2,55,KZ0294800KZT28600138,9.5024e+11,EURIKZKA,KZ,528657550
45,528657553,398,2017-01-01,136206.83,423,25,KZ,1,2,55,KZ0294800KZT28600138,9.5024e+11,EURIKZKA,KZ,528657553
73,528657556,398,2017-01-01,59613.06,421,25,KZ,1,2,55,KZ0294800KZT28600138,9.5024e+11,EURIKZKA,KZ,528657556
75,528657559,398,2017-01-01,29838.62,423,25,KZ,1,2,55,KZ0294800KZT28600138,9.5024e+11,EURIKZKA,KZ,528657559
1185,528744488,398,2017-01-02,171082.14,423,25,KZ,1,2,55,KZ0294800KZT28600138,9.5024e+11,EURIKZKA,KZ,528744488
1217,528744491,398,2017-01-02,27233.88,421,25,KZ,1,2,55,KZ0294800KZT28600138,9.5024e+11,EURIKZKA,KZ,528744491
1219,528744494,398,2017-01-02,481175.56,423,25,KZ,1,2,55,KZ0294800KZT28600138,9.5024e+11,EURIKZKA,KZ,528744494
1251,528744497,398,2017-01-02,28554.16,423,25,KZ,1,2,55,KZ0294800KZT28600138,9.5024e+11,EURIKZKA,KZ,528744497
2533,528850204,398,2017-01-03,129454.58,421,25,KZ,1,2,55,KZ0294800KZT28600138,9.5024e+11,EURIKZKA,KZ,528850204
2535,528850207,398,2017-01-03,81431.63,421,25,KZ,1,2,55,KZ0294800KZT28600138,9.5024e+11,EURIKZKA,KZ,528850207


In [107]:
a = t[["P_EKNPCODE", "P_CLIENTROLE", "P_BASEAMOUNT"]].pivot_table(columns=["P_EKNPCODE", "P_CLIENTROLE"], values=["P_BASEAMOUNT"], aggfunc=np.sum)

In [122]:
merged.head(100000).P_ACCOUNT.value_counts()

НЕ ОПРЕДЕЛЕН                                          14140
KZ619480003A01560161                                    106
KZ39948KZT27204000PE                                     66
KZ379480011A02000678                                     58
KZ589480019A02015913                                     54
KZ0294800KZT28600138                                     39
KZ249480016A02190205                                     35
KZ539480019A01866041                                     33
KZ769480004A02100714                                     33
KZ309480008A02081073                                     33
БЕЗ ИСПОЛЬЗОВАНИЯ СЧЕТА, ОТКРЫТОГО В АО ЕВРАЗИЙСКО       32
KZ759480018A00817803                                     30
KZ289480061A02078415                                     29
KZ259480013A02144938                                     28
KZ599480006A02076575                                     28
KZ559480621A02037394                                     28
KZ419480018A00032133                    

In [86]:
a[0] - pd.DateOffset(days=7)

Timestamp('2016-12-25 13:33:00')

In [62]:
to_take_off_op = ["ID", "P_CURRENCYCODE", "P_OPERATIONDATETIME", "P_BASEAMOUNT", "P_EKNPCODE", "P_DOCCATEGORY", ]
to_take_off_mem = ["P_COUNTRYCODE", "P_CLIENT_TYPE", "P_CLIENTROLE", "P_CLIENTKIND", "P_ACCOUNT", "P_CLIENTID",
                   "P_BANK", "P_BANKCOUNTRYCODE", "P_OPERATIONID"]

off_mem.head()

,ID,P_OPERATIONID,P_CLIENTID,P_BSCLIENTID,P_BANK_CLIENT,P_REGOPENDATE,P_COUNTRYCODE,P_CLIENT_TYPE,P_CLIENTROLE,P_CLIENTKIND,P_ACCOUNT,P_BSACCOUNT,P_BANK,P_BANKCOUNTRYCODE,P_BANKNAME,P_IPDL,P_DATE_INSERT,P_DATE_UPDATE,P_USERNAME,CHANGEDATE,P_SDP,P_ORGFORM,P_BANKCITY,P_OPER_DATE,P_OPERATIONDATE
0,529719568,529719567,9.5024e+11,6955,2,NaN,KZ,3,1,55,KZ6094800KZT17410144,1741,190501948,KZ,АО ЕВРАЗИЙСКИЙ БАНК,1,2017-01-06 09:12:09,NaN,LOADER,2017-01-06 09:12:09,NaN,NaN,АЛМАТЫ,2017-01-04 07:14:20,2017-01-04 00:00:00
1,529719569,529719567,9.5024e+11,88,2,2003-09-02 00:00:00,KZ,1,2,55,KZ8494800KZT28600117,2860,EURIKZKA,KZ,АО ЕВРАЗИЙСКИЙ БАНК,1,2017-01-06 09:12:09,NaN,LOADER,2017-01-06 09:12:09,NaN,28.0,АЛМАТЫ,2017-01-04 07:14:20,2017-01-04 00:00:00
2,529722751,529722750,9.5024e+11,88,2,2003-09-02 00:00:00,KZ,1,1,55,KZ5494800KZT59210074,5921,190501948,KZ,АО ЕВРАЗИЙСКИЙ БАНК,1,2017-01-06 09:12:36,NaN,LOADER,2017-01-06 09:12:36,NaN,28.0,АЛМАТЫ,2017-01-04 07:17:03,2017-01-04 00:00:00
3,529722752,529722750,9.5024e+11,88,2,2003-09-02 00:00:00,KZ,1,2,55,KZ3494800KZT49000020,4900,EURIKZKA,KZ,АО ЕВРАЗИЙСКИЙ БАНК,1,2017-01-06 09:12:36,NaN,LOADER,2017-01-06 09:12:36,NaN,28.0,АЛМАТЫ,2017-01-04 07:17:03,2017-01-04 00:00:00
4,529722754,529722753,9.5024e+11,6955,2,NaN,KZ,3,1,55,KZ3894800KZT17410152,1741,190501948,KZ,АО ЕВРАЗИЙСКИЙ БАНК,1,2017-01-06 09:12:36,NaN,LOADER,2017-01-06 09:12:36,NaN,NaN,АЛМАТЫ,2017-01-04 07:19:40,2017-01-04 00:00:00


In [56]:
off_op.P_BRANCH.value_counts(dropna=False)

0        2531918
7        1606665
0         772695
12        349100
9         270637
5         269758
3         254926
6         195298
6         187515
7         164336
4         157083
5         128138
12        125412
9         109239
15        105687
19        101059
10         96137
8          83297
3          78202
11         75017
14         72531
4          70266
13         68157
19         45645
15         42805
8          41625
10         41382
11         37973
17         32744
13         30923
14         30773
20         18105
17         15060
20          8818
WAY 4       3336
-1             1
Name: P_BRANCH, dtype: int64

In [23]:
susp_op[["P_SUSPICIOUSTYPECODE", "label"]].groupby("P_SUSPICIOUSTYPECODE").agg(["mean", "count", "sum"]).reset_index().sort_values(by=("label", "count"), ascending=False)

P_SUSPICIOUSTYPECODE     label                
                             mean   count     sum
11               1671.0  0.001761  257771   454.0
2                 511.0  0.034119   71808  2450.0
15               2011.0  0.000169   29660     5.0
3                 521.0  0.587091   15788  9269.0
4                 530.0  1.000000    6850  6850.0
9                 911.0  1.000000    4044  4044.0
14               1911.0  0.997144    3852  3841.0
17               7008.0  0.000398    2513     1.0
7                 633.0  1.000000    2389  2389.0
18               7013.0  0.000000    2096     0.0
6                 623.0  1.000000    1863  1863.0
21               7031.0  0.000000     747     0.0
20               7017.0  0.000000     521     0.0
0                 211.0  1.000000     451   451.0
13               1821.0  0.996923     325   324.0
10               1111.0  1.000000     300   300.0
8                 640.0  1.000000     242   242.0
5                 540.0  1.000000     108   108.0
1                 221.0  1.000000      94    94.0
22               7034.0  0.000000      61     0.0
16               2020.0  1.000000      28    28.0
12               1811.0  1.000000      19    19.0
19               7016.0  0.000000       5     0.0